In [ ]:
from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.figure_factory as ff
config={'showLink': False, 'displayModeBar': False}

init_notebook_mode(connected=True)

from electropy.charge import Charge
from electropy.volume import *

In [ ]:
# Calculate electric field

charge_val = 1e-7
position = 5
volume_dim = 10
z_plane = 0
field_scaling = 20
field_threshold = 15

charges = [Charge([0, -position, 0], -charge_val)]
charges.append(Charge([0, +position, 0], +charge_val))
h = 1

x, y = np.meshgrid(
    np.arange(-volume_dim, volume_dim + h, h),
    np.arange(-volume_dim, volume_dim + h, h)
)

# These two are "switched", not sure why numpy needs this. Needs clarification.
u = np.squeeze(field(charges, z_range=[z_plane, z_plane], h = h, type="potential", component='y'))
v = np.squeeze(field(charges, z_range=[z_plane, z_plane], h = h, type="potential", component='x'))

# Scaling and thresholding the field so that the arrows fit nicely in the plot
u=u/field_scaling
v=v/field_scaling
threshold = field_threshold

nan_mask = np.isnan(u) + np.isnan(v)
v[nan_mask] = 0
u[nan_mask] = 0

mask = (u**2+v**2)>threshold

v[mask] = 0
u[mask] = 0

In [ ]:
# Initialize plotly plot for electric field


quiver_data = ff.create_quiver(x, y, u, v,
                       scale=.25,
                       arrow_scale=0.3,
                       name='Electric field',
                       line=dict(width=1,
                                color = 'black'))

trace1=quiver_data.data[0]

In [ ]:
# Calculate potential

charge_val = 1e-7
position = 5
volume_dim = 10
z_plane = 0

charges = [Charge([0, -position, 0], -charge_val)]
charges.append(Charge([0, +position, 0], +charge_val))
h = 0.1

xi = np.arange(-volume_dim, volume_dim + h, h)
yi = np.arange(-volume_dim, volume_dim + h, h)
zi = potential(charges, z_range=[z_plane, z_plane], h = h)
zi=np.nan_to_num(zi)

In [ ]:
# Initialize plotly plot for potential

trace2 = go.Contour(
    x=xi,
    y=yi,
    z=np.squeeze(zi),
    ncontours=100,
    colorscale='RdBu',
    line=dict(smoothing=0.85),
    zmin=-600,
    zmax=600,
)

In [ ]:
# Plot superposition of figures

data = [trace1, trace2]

layout = go.Layout(
    width=700,
    height=700,
    xaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        ticks='',
        showticklabels=False
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        ticks='',
        showticklabels=False
    )
)

fig = dict(data=data, layout=layout)

iplot(fig)